# RDKit pandas support
This is a quick tutorial will show some of the ways you can use RDKit together with pandas.

In [1]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th{border: 1px 
black solid ! important;color:black !important;}

In [2]:
from __future__ import print_function
%matplotlib inline

In [3]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

# The next line is commented out 
# because GitHub does not render svg's embedded in notebooks
IPythonConsole.ipython_useSVG=False

Load the table of drugs (downloaded from [ChEMBL](https://www.ebi.ac.uk/chembl/) )

In [1]:
df = pd.read_csv('../data/chembl_drugs.txt.gz', sep='\t')

NameError: name 'pd' is not defined

In [ ]:
[str(x) for x in df.columns]

In [ ]:
len(df)

Keep only compounds with SMILES,that respect Ro5 and are on the market

In [2]:
df = df[df['CANONICAL_SMILES'].notnull() & # Keep cpds with SMILES
        (df['RULE_OF_FIVE'] == 'Y') & # that respect Ro5
        (df['DEVELOPMENT_PHASE'] == 4)] # are on the market

NameError: name 'df' is not defined

In [ ]:
len(df)

Table contains CANONICAL_SMILES with SMILES which we can convert to RDKit molecules (default name ROMol)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df, smilesCol='CANONICAL_SMILES')

Remove rows where RDKit failed to generate a molecule from SMILES

In [3]:
df = df[~df['ROMol'].isnull()]

NameError: name 'df' is not defined

Extract a name from SYNONYMS column by applying a row-wise operation

In [ ]:
df['name'] = df.apply(lambda x: x['SYNONYMS'].split('(')[0] if type(x['SYNONYMS']) is str else None, axis=1)

Depict first 8 mols

In [ ]:
PandasTools.FrameToGridImage(df.head(8), legendsCol='name', molsPerRow=4)

Calculate some descriptors and visualize distributions

In [ ]:
from rdkit.Chem import Descriptors

In [ ]:
df['MW'] = df['ROMol'].map(Descriptors.MolWt)
df['logP'] = df['ROMol'].map(Descriptors.MolLogP)

In [ ]:
#doctest: IGNORE
df['MW'].hist();

In [ ]:
#doctest: IGNORE
df['logP'].hist();

Do a subsructure search on a dataframe

In [ ]:
query = Chem.MolFromSmarts('Nc1ccc(S(=O)(=O)-[*])cc1')

In [ ]:
#doctest: IGNORE
Chem.Draw.MolToImage(query, kekulize=False)

In [ ]:
len(df[df['ROMol'] >= query])

RDKit by defaults highlights the matched substructures

In [ ]:
df[df['ROMol'] >= query][['SYNONYMS', 'ROMol']].head()

Save the table as SD file

In [ ]:
PandasTools.WriteSDF(df, '../data/approved_drugs.sdf', idName='CHEMBL_ID', properties=df.columns)

Tutorial author: Samo Turk, Jan. 2017